# Create batch endpoint

In [12]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import BatchEndpoint, BatchDeployment, BatchRetrySettings
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import BatchDeploymentOutputAction

Parameters

In [13]:
batch_endpoint_name = "dbcc-batch"
batch_deployment_name = "DBCC-batch-endpoint-deployment"
compute_name = "paolt-run-cpu-vm"
environment_name = "DataBook-Env"
environment_ver = "3"
model_name = "dbcc"
model_version = "1"

In [14]:
environment_reference = f"{environment_name}:{environment_ver}"
model_reference = f"{model_name}:{model_version}"
environment_reference, model_reference

('DataBook-Env:3', 'dbcc:1')

In [15]:
ml_client = MLClient.from_config(
    credential=DefaultAzureCredential())
ml_client

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/paolt-dev-cpu-vm/code/Users/paolt/databook_v1/.azureml/config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f3852e82b20>,
         subscription_id=fadb1e32-9c96-4180-be9d-1811f4687cca,
         resource_group_name=paolt-ml-v2,
         workspace_name=paolt-ml-v2)

In [16]:
try:
    end_point = ml_client.batch_endpoints.get(name=batch_endpoint_name)
    print(f"Endpoint {batch_endpoint_name} already exists")
except:
    endpoint = BatchEndpoint(
        name=batch_endpoint_name,
        description="DBCC Batch end-point",
    )
    ml_client.begin_create_or_update(endpoint)

Endpoint dbcc-batch already exists


In [17]:
# create a batch deployment
# we should pair this with the publication of a model - this requires to use the model_path pipeline's output
deployment = BatchDeployment(
    name=batch_deployment_name,
    endpoint_name=batch_endpoint_name,
    model=model_reference,
    code_path="./scripts",
    scoring_script="batch-scoring-script.py",
    environment=environment_reference,
    compute=compute_name,
    instance_count=1,
    max_concurrency_per_instance=1,
    mini_batch_size=10,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=30),
    logging_level="info",
)

In [18]:
ml_client.begin_create_or_update(deployment)

BatchDeployment({'endpoint_name': 'dbcc-batch', 'type': None, 'name': 'dbcc-batch-endpoint-deployment', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/fadb1e32-9c96-4180-be9d-1811f4687cca/resourceGroups/paolt-ml-v2/providers/Microsoft.MachineLearningServices/workspaces/paolt-ml-v2/batchEndpoints/dbcc-batch/deployments/dbcc-batch-endpoint-deployment', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/paolt-dev-cpu-vm/code/Users/paolt/databook_v1/mlops', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f3852ea7550>, 'model': '/subscriptions/fadb1e32-9c96-4180-be9d-1811f4687cca/resourceGroups/paolt-ml-v2/providers/Microsoft.MachineLearningServices/workspaces/paolt-ml-v2/models/dbcc/versions/1', 'code_configuration': <azure.ai.ml.entities._deployment.code_configuration.CodeConfiguration object at 0x7f3852e95c40>, 'environment': '/subscriptions/fadb1e32-9c96-4180-be9d-1811f4687cca/resour